In [ ]:
import pandas as pd
import pyodbc
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Conexión al servidor
server = '10.54.200.90'
username = 'eettlin'
password = 'lOc4l_eXt$24;'
driver = '{ODBC Driver 17 for SQL Server}'

conn_str = (
    f'DRIVER={driver};'
    f'SERVER={server};'
    f'UID={username};'
    f'PWD={password};'
)

conn = pyodbc.connect(conn_str)

<h3>1. Indicadores de inventario </h3>

In [ ]:
# 1. Cobertura de Stock por C_RUBRO (Categoría)
query = f"""
SELECT 
    CAST(A.C_RUBRO AS VARCHAR) AS C_RUBRO,
    R.Q_DIAS_STOCK
FROM [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
INNER JOIN [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A
    ON A.C_ARTICULO = S.C_ARTICULO
INNER JOIN [DIARCOP001].[DiarcoP].[dbo].[T710_ESTADIS_REPOSICION] R
    ON R.C_ARTICULO = S.C_ARTICULO
    AND R.C_SUCU_EMPR = S.C_SUCU_EMPR
WHERE 
    S.M_HABILITADO_SUCU = 'S'
    AND A.M_BAJA = 'N'
    AND R.Q_DIAS_STOCK IS NOT NULL;
"""

# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()
df['C_RUBRO'] = df['C_RUBRO'].astype(float).astype(int).astype(str)
# Agrupar y calcular cobertura promedio
df_cobertura_rubro = df.groupby('C_RUBRO', as_index=False)['Q_DIAS_STOCK'].mean().round(2)

# Ordenar de mayor a menor
df_cobertura_rubro = df_cobertura_rubro.sort_values(by='Q_DIAS_STOCK', ascending=False)

# Asignar color según umbrales de días de stock
df_cobertura_rubro['color'] = df_cobertura_rubro['Q_DIAS_STOCK'].apply(
    lambda x: 'red' if x < 10 else ('green' if x > 30 else 'orange')
)

# --- 2. Crear gráfico ---
fig_rubro = px.bar(
    df_cobertura_rubro,
    x='C_RUBRO',
    y='Q_DIAS_STOCK',
    color='color',  # Color automático según umbrales
    color_discrete_map={'red': 'red', 'orange': 'orange', 'green': 'green'},
    title='Cobertura de Stock Promedio por Rubro (Categoría)',
    labels={'C_RUBRO': 'Rubro', 'Q_DIAS_STOCK': 'Días de Stock'}
)

# Mostrar etiquetas arriba de cada barra
fig_rubro.update_traces(
    texttemplate='%{y:.1f}', 
    textposition='inside',
    textfont_size=14
)

# Línea horizontal objetivo (por ejemplo 20 días)
fig_rubro.add_shape(
    type='line',
    x0=-0.5,
    x1=len(df_cobertura_rubro)-0.5,
    y0=20,
    y1=20,
    line=dict(color="blue", width=2, dash="dash"),
    xref='x', yref='y'
)

# Mejoras visuales
fig_rubro.update_layout(
    height=500,
    width=900,
    xaxis_title='Rubro',
    yaxis_title='Días de Stock',
    showlegend=False,  # Ocultar leyenda de color si querés
    margin=dict(l=20, r=20, t=60, b=50),
    title_x=0.5  # Centrar título
)

fig_rubro.show()


In [ ]:
# 2. Artículos con Stock Crítico (Tarjeta Indicator)
#IMPORTANTE: es un numero alto por que no distingue por sucursal
query = f"""
SELECT 
    S.C_ARTICULO,
    ST.Q_UNID_ARTICULO AS Q_STOCK_UNIDADES
FROM 
    [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A
    ON A.C_ARTICULO = S.C_ARTICULO
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T060_STOCK] ST
    ON ST.C_ARTICULO = S.C_ARTICULO 
    AND ST.C_SUCU_EMPR = S.C_SUCU_EMPR
WHERE 
    S.M_HABILITADO_SUCU = 'S'
    AND A.M_BAJA = 'N'
    AND ST.Q_UNID_ARTICULO IS NOT NULL
"""

# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

# Definimos nivel de seguridad
nivel_seguridad = 10

# Contar los artículos críticos
cantidad_stock_critico = df[df['Q_STOCK_UNIDADES'] < nivel_seguridad].shape[0]

# Crear la tarjeta
fig_stock_critico = go.Figure(go.Indicator(
    mode="number",
    value=cantidad_stock_critico,
    title={"text": "Artículos con Stock Crítico (< 20 unidades)"},
    number={"valueformat": "d", "font": {"size": 70}}
))

fig_stock_critico.update_layout(height=300)
fig_stock_critico.show()

In [ ]:
# 3. Articulos con sobre stock
query = f"""
SELECT 
    S.C_ARTICULO,
    S.[C_SUCU_EMPR],
    ST.Q_UNID_ARTICULO AS Q_STOCK_UNIDADES
FROM 
    [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A
    ON A.C_ARTICULO = S.C_ARTICULO
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T060_STOCK] ST
    ON ST.C_ARTICULO = S.C_ARTICULO 
    AND ST.C_SUCU_EMPR = S.C_SUCU_EMPR
WHERE 
    S.M_HABILITADO_SUCU = 'S'
    AND A.M_BAJA = 'N'
    AND ST.Q_UNID_ARTICULO IS NOT NULL
"""

# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

df['C_ARTICULO'] = df['C_ARTICULO'].astype(float).astype(int).astype(str)
df['C_SUCU_EMPR'] = df['C_SUCU_EMPR'].astype(float).astype(int).astype(str)
# Definir nivel óptimo para sobrestock
nivel_optimo = 1500

# Filtrar artículos en sobrestock
df_sobrestock = df[df['Q_STOCK_UNIDADES'] > nivel_optimo].copy()
cantidad_articulos = df_sobrestock['C_ARTICULO'].nunique()

# Ordenar por cantidad de unidades en stock (mayor primero)
df_sobrestock = df_sobrestock.sort_values(by='Q_STOCK_UNIDADES', ascending=False)

# Tomar solo los 10 artículos con más sobrestock
df_sobrestock_top10 = df_sobrestock.head(10)

# Crear columna combinada Artículo + Sucursal
df_sobrestock_top10['Articulo_Sucursal'] = df_sobrestock_top10['C_ARTICULO'].astype(str) + " - " + df_sobrestock_top10['C_SUCU_EMPR'].astype(str)

# Graficar
fig_sobrestock_top10 = px.bar(
    df_sobrestock_top10,
    x='Articulo_Sucursal',
    y='Q_STOCK_UNIDADES',
    title='Top 10 Artículos con Mayor SobreStock (> 300 unidades)',
    labels={'Articulo_Sucursal': 'Artículo - Sucursal', 'Q_STOCK_UNIDADES': 'Unidades en Stock'},
)

# Agregar texto del total como anotación
fig_sobrestock_top10.add_annotation(
    text=f"Total articulos en sobrestock : {cantidad_articulos}",
    xref="paper", yref="paper",
    x=0.5, y=1.10,  # Centrado arriba del gráfico
    showarrow=False,
    font=dict(size=16, color="black")
)

fig_sobrestock_top10.update_layout(
    xaxis_title='Artículo - Sucursal',
    yaxis_title='Unidades en Stock',
    xaxis_tickangle=45,
    bargap=0.2
)

fig_sobrestock_top10.show()

In [ ]:
# 4. Rotacion de inventario por sucursal
query = f"""
SELECT 
    S.C_ARTICULO,
    R.Q_VENTA_30_DIAS + R.Q_VENTA_15_DIAS AS Q_VENTA_ACUM_30,
    ST.Q_UNID_ARTICULO AS Q_STOCK_UNIDADES
FROM 
    [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A
    ON A.C_ARTICULO = S.C_ARTICULO
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T710_ESTADIS_REPOSICION] R
    ON R.C_ARTICULO = S.C_ARTICULO
    AND R.C_SUCU_EMPR = S.C_SUCU_EMPR
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T060_STOCK] ST
    ON ST.C_ARTICULO = S.C_ARTICULO 
    AND ST.C_SUCU_EMPR = S.C_SUCU_EMPR
WHERE 
    S.M_HABILITADO_SUCU = 'S'
    AND A.M_BAJA = 'N'
    AND R.Q_VENTA_30_DIAS IS NOT NULL
    AND R.Q_VENTA_15_DIAS IS NOT NULL
    AND ST.Q_UNID_ARTICULO IS NOT NULL;
"""

# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

"""df['C_RUBRO'] = df['C_RUBRO'].astype(float).astype(int).astype(str)
df['C_ARTICULO'] = df['C_ARTICULO'].astype(float).astype(int).astype(str)
df['C_SUCU_EMPR'] = df['C_SUCU_EMPR'].astype(float).astype(int).astype(str)"""
df['C_ARTICULO'] = df['C_ARTICULO'].astype(float).astype(int).astype(str)

# Calcular rotación de inventario por artículo
df['rotacion_inventario'] = df['Q_VENTA_ACUM_30'] / df['Q_STOCK_UNIDADES']

# Limpiar valores infinitos o nulos
df['rotacion_inventario'].replace([float('inf'), float('-inf')], None, inplace=True)
df['rotacion_inventario'].fillna(0, inplace=True)

# --- 2. Calcular eficiencia promedio ---
eficiencia_promedio = df['rotacion_inventario'].mean().round(4)
objetivo_rotacion = 0.6  # 1 rotación cada 30 días, por ejemplo
# --- 3. Crear indicador ---
fig_eficiencia = go.Figure(go.Indicator(
    mode="number+delta",
    value=eficiencia_promedio,
    number={
        "valueformat": ".4f",
        "font": {"size": 60}
    },
    title={
        "text": f"Eficiencia de Inventarios<br><span style='font-size:20px'>Rotación promedio (Objetivo: {objetivo_rotacion:.2f})</span>"
    },
    delta={
        "reference": objetivo_rotacion,
        "relative": False,
        "position": "top",
        "increasing": {"color": "green"},
        "decreasing": {"color": "red"}
    }
))

fig_eficiencia.update_layout(
    height=300,
    margin=dict(l=20, r=20, t=40, b=20)
)

fig_eficiencia.show()


<h3>2. Indicadores de Órdenes de Compra </h3>

In [ ]:
from plotly.subplots import make_subplots
# Traer todas las tablas del linked server
query = f"""
SELECT 
    cab.U_SUFIJO_OC,
    cab.U_DIAS_LIMITE_ENTREGA,
    cab.C_PROVEEDOR,
    cab.C_SUCU_COMPRA,
    pend.C_SITUAC,
    cab.F_ALTA_SIST,
    pend.F_EMISION,
    pend.F_ENTREGA,
    DATEADD(DAY, cab.U_DIAS_LIMITE_ENTREGA, cab.F_EMISION) AS FECHA_ACORDADA,
    cab.I_TOTAL_OC
FROM [DIARCOP001].[DiarcoP].[dbo].[T080_OC_CABE] cab
INNER JOIN [DIARCOP001].[DiarcoP].[dbo].[T080_OC_CABE_PEND_AUTORIZACION] pend
    ON cab.[U_SUFIJO_OC] = pend.[U_SUFIJO_OC];
"""

# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

"""df['C_RUBRO'] = df['C_RUBRO'].astype(float).astype(int).astype(str)
df['C_ARTICULO'] = df['C_ARTICULO'].astype(float).astype(int).astype(str)
df['C_SUCU_EMPR'] = df['C_SUCU_EMPR'].astype(float).astype(int).astype(str)"""
# Convertir fechas
df['F_EMISION'] = pd.to_datetime(df['F_EMISION'], errors='coerce')
df['F_ENTREGA'] = pd.to_datetime(df['F_ENTREGA'], errors='coerce')
df['F_ALTA_SIST'] = pd.to_datetime(df['F_ALTA_SIST'], errors='coerce')
df['FECHA_ACORDADA'] = pd.to_datetime(df['FECHA_ACORDADA'], errors='coerce')

# --- Filtrar solo órdenes en estado pendiente para los KPIs ---
df_pendientes = df[df['C_SITUAC'] == 1]

# --- 1. Órdenes Pendientes de Aprobación ---
cantidad_ordenes_pendientes = df_pendientes.shape[0]
valor_total_ordenes_pendientes = df_pendientes['I_TOTAL_OC'].sum()

# --- 2. Órdenes Atrasadas ---
df['atrasada'] = df['F_ENTREGA'] > df['FECHA_ACORDADA']

# % de órdenes atrasadas
porcentaje_atrasadas = df['atrasada'].mean() * 100
# Convertir valor total pendiente a millones
valor_total_millones = valor_total_ordenes_pendientes / 1_000_000

# Crear figura con subplots de indicadores
fig = make_subplots(
    rows=1, cols=4,
    specs=[[{"type": "indicator"}]*3],
    subplot_titles=(
        "Órdenes Pendientes de aprobacion",
        "Valor Total Pendiente (MM)",
        "% Órdenes Atrasadas"
    )
)

# 1. Número de órdenes pendientes
fig.add_trace(go.Indicator(
    mode="number",
    value=cantidad_ordenes_pendientes,
    number={"valueformat": "d", "font": {"size": 50}},
), row=1, col=1)

# 2. Valor total de órdenes pendientes (en millones)
fig.add_trace(go.Indicator(
    mode="number",
    value=valor_total_millones,
    number={"prefix": "$", "suffix": "M", "valueformat": ".1f", "font": {"size": 50}},
), row=1, col=2)

# 3. % de órdenes atrasadas
fig.add_trace(go.Indicator(
    mode="number",
    value=porcentaje_atrasadas,
    number={"suffix": "%", "valueformat": ".1f", "font": {"size": 50}},
), row=1, col=3)

# --- Ajustes finales del layout ---
fig.update_layout(
    height=400,    # MÁS espacio vertical
    width=1400,
    margin=dict(l=50, r=50, t=120, b=20),  # MÁS espacio arriba (t=120)
    title={
        'text': f"<b style='background-color:#f0f0f0; padding:10px; border-radius:10px;'>Resumen de Órdenes Pendientes de Aprobación</b>",
        'x': 0.5,
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20}
    },
)

fig.show()

In [ ]:
query = f"""
SELECT 
    pend.U_SUFIJO_OC,
    pend.F_EMISION,
    hist.F_SITUAC AS F_SITUAC_CAMBIO_ESTADO
FROM 
    [DIARCOP001].[DiarcoP].[dbo].[T080_OC_CABE_PEND_AUTORIZACION] pend
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T080_OC_PEND_AUTORIZACION_ESTADOS_HISTORIA] hist
    ON pend.[U_SUFIJO_OC] = hist.[U_SUFIJO_OC]
    AND hist.C_SITUAC_ANTERIOR = 1
    AND hist.C_SITUAC_NUEVA = 2
WHERE pend.F_EMISION < hist.F_SITUAC
"""

# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

# Convertir fechas
df['F_EMISION'] = pd.to_datetime(df['F_EMISION'])
df['F_SITUAC_CAMBIO_ESTADO'] = pd.to_datetime(df['F_SITUAC_CAMBIO_ESTADO'])

# Calcular diferencia en días
df['tiempo_gestion_dias'] = (df['F_SITUAC_CAMBIO_ESTADO'] - df['F_EMISION']).dt.days

# Mantener solo tiempos positivos
df = df[df['tiempo_gestion_dias'] >= 0]

# Tiempo promedio de gestión
tiempo_promedio_gestion = df['tiempo_gestion_dias'].mean().round(2)

# --- Color dinámico según valor ---
if tiempo_promedio_gestion < 2:
    color_indicator = "green"
elif tiempo_promedio_gestion <= 5:
    color_indicator = "orange"
else:
    color_indicator = "red"

# --- Crear tarjeta Plotly ---
fig = go.Figure(go.Indicator(
    mode="number",
    value=tiempo_promedio_gestion,
    number={
        "suffix": " días", 
        "valueformat": ".1f", 
        "font": {"size": 60, "color": color_indicator}
    },
    title={
        "text": "<b>Tiempo Promedio de Gestión de OC</b>",
        "font": {"size": 24}
    },
    domain={"x": [0, 1], "y": [0, 1]}  
))

fig.update_layout(
    height=300,
    width=600,
    margin=dict(l=40, r=40, t=80, b=20), 
    paper_bgcolor="white"
)

fig.show()

 <h3>3. Indicadores de Proveedores</h3>

In [ ]:
#Cumplimiento de proveedores OTIF
query = f"""
SELECT 
    cab.C_PROVEEDOR,
    cab.C_OC,
    cab.U_PREFIJO_OC,
    cab.U_SUFIJO_OC,
    cab.F_EMISION,
    cab.F_ENTREGA,
    cab.C_SITUAC,
    cab.U_DIAS_LIMITE_ENTREGA,
    DATEADD(DAY, cab.U_DIAS_LIMITE_ENTREGA, cab.F_ENTREGA) AS FECHA_ACORDADA,
    det.M_CUMPLIDA_PARCIAL,
    YEAR(cab.F_ENTREGA) AS ANIO_ENTREGA,
    MONTH(cab.F_ENTREGA) AS MES_ENTREGA
FROM 
    [DIARCOP001].[DiarcoP].[dbo].[T080_OC_CABE] cab
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T081_OC_DETA] det
ON 
    cab.C_OC = det.C_OC 
    AND cab.U_PREFIJO_OC = det.U_PREFIJO_OC
    AND cab.U_SUFIJO_OC = det.U_SUFIJO_OC
WHERE 
    YEAR(cab.F_ENTREGA) = 2025
"""

# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()


# Convertir fechas
df['F_ENTREGA'] = pd.to_datetime(df['F_ENTREGA'])
df['FECHA_ACORDADA'] = pd.to_datetime(df['FECHA_ACORDADA'])

# --- FILTRAR las OC que aún no llegaron a la fecha pactada ---
mask_pendiente_y_en_tiempo = (df['FECHA_ACORDADA'] >= pd.Timestamp.now())
df = df[~mask_pendiente_y_en_tiempo]

# --- Calcular si estuvo en tiempo y completo ---
df['en_tiempo'] = df['F_ENTREGA'] <= (df['FECHA_ACORDADA'] + pd.Timedelta(days=1))

# AHORA COMPLETO = (entregado completo en detalle) Y (C_SITUAC = 2 en cabecera)
df['completo'] = (df['M_CUMPLIDA_PARCIAL'] == 'N') & (df['C_SITUAC'] == 2)

# --- Agrupar por OC ---
df_oc = df.groupby('U_SUFIJO_OC').agg({
    'en_tiempo': 'all',
    'completo': 'all',
    'ANIO_ENTREGA': 'first',
    'MES_ENTREGA': 'first'
}).reset_index()

# --- Calcular OTIF ---
df_oc['otif'] = df_oc['en_tiempo'] & df_oc['completo']

# --- Agrupar por Mes ---
otif_por_mes = df_oc.groupby(['ANIO_ENTREGA', 'MES_ENTREGA'])['otif'].mean().reset_index()
otif_por_mes['otif'] = otif_por_mes['otif'] * 100  # Convertir a %

# Formato para el gráfico
otif_por_mes['periodo'] = pd.to_datetime(
    otif_por_mes['ANIO_ENTREGA'].astype(str) + '-' + otif_por_mes['MES_ENTREGA'].astype(str).str.zfill(2) + '-01'
)
otif_por_mes['nombre_mes'] = otif_por_mes['MES_ENTREGA'].apply(lambda x: calendar.month_name[x])

#GRAFICO
fig = px.bar(
    otif_por_mes,
    x='nombre_mes',
    y='otif',
    text=otif_por_mes['otif'].apply(lambda x: f"{x:.1f}%"),
    title=f'Cumplimiento OTIF Mensual<br>',
    labels={'otif': 'Cumplimiento (%)', 'nombre_mes': 'Mes'},
)

fig.update_traces(
    textposition='inside',  # Coloca el texto dentro de la barra
    textfont=dict(size=20, color="white"),  # Aumenta el tamaño del texto y lo hace blanco para mejor visibilidad
    marker_color='rgba(0, 180, 220, 0.7)'  # Mantiene el color de las barras
)

fig.update_layout(
    height=450,
    width=800,
    yaxis_tickformat=".0%",
    xaxis_title='Mes',
    yaxis_title='Cumplimiento (%)',
    title_x=0.5,
    margin=dict(l=20, r=20, t=70, b=50)
)

fig.show()

In [ ]:
#Notas de credito pendientes
query = f"""
SELECT cab.U_COBRO_SUFIJO,
cab.F_ALTA_SIST,
det.I_DOC
FROM [DIARCOP001].[DiarcoP].[dbo].[T220_COBRO_CABE] cab
INNER JOIN [DIARCOP001].[DiarcoP].[dbo].[T220_COBRO_DETA] det
ON cab.U_COBRO_SUFIJO = det.U_COBRO_SUFIJO
WHERE det.C_DOC = 2
"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()
# --- AGRUPACIÓN POR U_COBRO_SUFIJO ---
df_grouped = df.groupby('U_COBRO_SUFIJO', as_index=False)['I_DOC'].sum()

# --- INDICADORES ---
total_notas_credito = df_grouped.shape[0]
total_importe_millones = round(df_grouped['I_DOC'].sum() / 1_000_000, 1)*-1

# --- GRÁFICO CON PLOTLY ---
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode="number",
    value=total_notas_credito,
    title={"text": "Cantidad de Notas de Crédito"},
    domain={'x': [0, 0.5], 'y': [0, 1]}
))

fig.add_trace(go.Indicator(
    mode="number",
    value=total_importe_millones,
    number={"suffix": " MM"},
    title={"text": "Total Importe de NC"},
    domain={'x': [0.5, 1], 'y': [0, 1]}
))

fig.update_layout(
    grid={'rows': 1, 'columns': 2},
    title="Indicadores de Notas de Crédito por U_COBRO_SUFIJO"
)

fig.show()

In [ ]:
#Deuda de Proveedores: Valor total de facturas pendientes por pagar
query = f"""
SELECT 
    SUM(I_TOTAL_COMP) AS TOTAL_DEUDA_PROVEEDORES
FROM [DIARCOP001].[DiarcoP].[dbo].[T021_PROV_COMPROB]
WHERE C_SITUAC = 1
  AND F_SITUAC >= '2024-01-01';
"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

total_deuda = df.iloc[0]['TOTAL_DEUDA_PROVEEDORES']

# Convertir a millones y redondear
total_deuda_mm = round(total_deuda / 1_000_000, 1)

# Crear tarjeta visual
fig = go.Figure(go.Indicator(
    mode="number",
    value=total_deuda_mm,
    number={'prefix': "$ ", 'suffix': " MM", 'font': {'size': 48}},
    title={'text': "Deuda Total a Proveedores", 'font': {'size': 20}}
))

fig.update_layout(
    height=200,
    margin=dict(t=40, b=0, l=0, r=0)
)

fig.show()

In [ ]:
#Plazo de entrega promedio
query = f"""
SELECT 
    cab.C_OC,
    cab.U_PREFIJO_OC,
    cab.U_SUFIJO_OC,
    cab.U_DIAS_LIMITE_ENTREGA,
    DATEADD(DAY, cab.U_DIAS_LIMITE_ENTREGA, cab.F_EMISION) AS FECHA_LIMITE,
    DATEDIFF(DAY, cab.F_EMISION, cab.F_ENTREGA) AS DEMORA,
    cab.C_PROVEEDOR,
    prov.N_PROVEEDOR,
    cab.I_TOTAL_OC,
    cab.C_SITUAC,
    cab.F_SITUAC,
    cab.F_EMISION,
    cab.F_ENTREGA
FROM 
    [DIARCOP001].[DiarcoP].[dbo].[T080_OC_CABE] cab
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T020_PROVEEDOR] prov
    ON cab.C_PROVEEDOR = prov.C_PROVEEDOR
WHERE 
    prov.M_ACTIVO = 'S'
    AND cab.F_ENTREGA >= DATEADD(MONTH, -12, GETDATE())
    AND cab.U_DIAS_LIMITE_ENTREGA > 0
    AND cab.F_EMISION < cab.F_ENTREGA
    AND cab.C_SITUAC = 2
	AND cab.F_ENTREGA <= GETDATE()
"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()
# Agrupar por nombre del proveedor y calcular promedio de demora
df_proveedor_demora = df.groupby('N_PROVEEDOR', as_index=False)['DEMORA'].mean()
# Redondear
df_proveedor_demora['DEMORA'] = df_proveedor_demora['DEMORA'].round(1)

# Top 10
df_top10_proveedores = df_proveedor_demora.sort_values(by='DEMORA', ascending=False).head(10)
# Crear tabla tipo dinámica
fig = go.Figure(data=[go.Table(
    header=dict(values=['Proveedor', 'Tiempo Promedio de Entrega (días)'],
                fill_color='paleturquoise',
                align='center',
                font=dict(size=16)),
    cells=dict(values=[
        df_top10_proveedores['N_PROVEEDOR'],
        df_top10_proveedores['DEMORA']
    ],
        fill_color='lavender',
        align='center',
        format=["", ".1f"],  # Formato de un decimal en DEMORA
        font=dict(size=14))
)])

fig.update_layout(
    title='<b>Top 10 Proveedores con Mayor Tiempo Promedio de Entrega</b>',
    title_x=0.5,
    height=500,
    width=700,
    margin=dict(l=20, r=20, t=70, b=20)
)

fig.show()

<h3>4. Indicadores de Desempeño Personal</h3>

In [ ]:
#Eficiencia de Sugerencias: % de artículos sugeridos que se convierten en pedidos efectivos
query = f"""
SELECT 
    CAST(100.0 * 
        SUM(CASE 
                WHEN Q_BULTOS_SUGERIDOS = Q_BULTOS_PROV_PED 
                THEN 1 ELSE 0 
            END) 
        / COUNT(*) AS DECIMAL(5,2)
    ) AS PORCENTAJE_COINCIDENCIA
FROM [DIARCOP001].[DiarcoP].[dbo].[T081_OC_DETA]
WHERE Q_BULTOS_SUGERIDOS IS NOT NULL 
  AND Q_BULTOS_PROV_PED IS NOT NULL;

"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()
# Extraer el valor como número
porcentaje = df.iloc[0]['PORCENTAJE_COINCIDENCIA']

# Crear tarjeta de indicador
fig = go.Figure(go.Indicator(
    mode="number",
    value=porcentaje,
    number={'suffix': " %", 'font': {'size': 48}},
    title={'text': "Eficiencia de sugerencias", 'font': {'size': 18}}
))

fig.update_layout(
    height=200,
    margin=dict(t=40, b=0, l=0, r=0)
)

fig.show()

In [ ]:
# Impacto de las órdenes realizadas en la cobertura del surtido. 
query = f"""
WITH OC_ORDENADAS AS (
    SELECT 
        det.C_ARTICULO,
        cab.C_SUCU_DESTINO AS C_SUCU_EMPR,
        det.Q_BULTOS_PROV_PED-- * det.Q_FACTOR_PROV_PED AS Q_UNIDADES_ORDENADAS
    FROM DIARCOP001.DiarcoP.dbo.T081_OC_DETA det
    INNER JOIN DIARCOP001.DiarcoP.dbo.T080_OC_CABE cab
        ON det.C_OC = cab.C_OC
        AND det.U_PREFIJO_OC = cab.U_PREFIJO_OC
        AND det.U_SUFIJO_OC = cab.U_SUFIJO_OC
    WHERE cab.C_SITUAC = 2
),
NECESIDAD_SUCU AS (
    SELECT 
        C_ARTICULO,
        C_SUCU_EMPR,
        Q_15DIASVTA_A_ULT_ING_STOCK
    FROM DIARCOP001.DiarcoP.dbo.T051_ARTICULOS_SUCURSAL
    WHERE M_HABILITADO_SUCU = 'S'
      AND Q_15DIASVTA_A_ULT_ING_STOCK > 0
)

SELECT 
    CAST(100.0 * 
        SUM(CASE 
                WHEN oc.Q_BULTOS_PROV_PED >= art.Q_15DIASVTA_A_ULT_ING_STOCK THEN 1 
                ELSE 0 
            END) 
        / COUNT(*) AS DECIMAL(5,2)
    ) AS CONTRIBUCION_FILL_RATE_PORCENTAJE
FROM NECESIDAD_SUCU art
LEFT JOIN OC_ORDENADAS oc
    ON oc.C_ARTICULO = art.C_ARTICULO 
    AND oc.C_SUCU_EMPR = art.C_SUCU_EMPR;
"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()
contribucion_fill_rate = df.iloc[0]['CONTRIBUCION_FILL_RATE_PORCENTAJE']
# Crear tarjeta de porcentaje
fig = go.Figure(go.Indicator(
    mode="number",
    value=contribucion_fill_rate,
    number={'suffix': " %", 'font': {'size': 48}},
    title={'text': "Contribución al Fill Rate", 'font': {'size': 20}}
))

fig.update_layout(
    height=200,
    margin=dict(t=40, b=0, l=0, r=0)
)

fig.show()

In [ ]:
#Indicador de rentabilidad
query = f"""
WITH OC_DETALLE AS (
    SELECT 
        cab.C_COMPRADOR,
        det.C_ARTICULO,
        MAX(det.I_PRECIO_COMPRA) AS ULTIMO_COSTO
    FROM DIARCOP001.DiarcoP.dbo.T080_OC_CABE cab
    INNER JOIN DIARCOP001.DiarcoP.dbo.T081_OC_DETA det
        ON cab.C_OC = det.C_OC
        AND cab.U_PREFIJO_OC = det.U_PREFIJO_OC
        AND cab.U_SUFIJO_OC = det.U_SUFIJO_OC
    WHERE cab.F_EMISION >= DATEADD(DAY, -30, GETDATE())
    GROUP BY cab.C_COMPRADOR, det.C_ARTICULO
)

SELECT 
    oc.C_COMPRADOR,
    art.C_ARTICULO,
    art.I_PRECIO_VTA,
    oc.ULTIMO_COSTO,
    CASE 
        WHEN art.I_PRECIO_VTA > 0 THEN 
            ROUND(((art.I_PRECIO_VTA - oc.ULTIMO_COSTO) / art.I_PRECIO_VTA) * 100.0, 2)
        ELSE NULL
    END AS MARGEN_BRUTO_PORCENTAJE
FROM OC_DETALLE oc
INNER JOIN DIARCOP001.DiarcoP.dbo.T051_ARTICULOS_SUCURSAL art
    ON oc.C_ARTICULO = art.C_ARTICULO
WHERE art.M_HABILITADO_SUCU = 'S'
  AND art.I_PRECIO_VTA IS NOT NULL
  AND oc.ULTIMO_COSTO IS NOT NULL;
"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()
df['C_COMPRADOR'] = df['C_COMPRADOR'].astype(float).astype(int).astype(str)
# FILTRO OPCIONAL: excluir márgenes negativos o cero
df = df[df['MARGEN_BRUTO_PORCENTAJE'] > 0]

# Calcular promedio por comprador
df_avg = df.groupby('C_COMPRADOR', as_index=False)['MARGEN_BRUTO_PORCENTAJE'].mean()

df_avg['MARGEN_BRUTO_PORCENTAJE'] = df_avg['MARGEN_BRUTO_PORCENTAJE'].round(1)

# Crear gráfico de columnas (vertical)
fig = px.bar(
    df_avg,
    x='C_COMPRADOR',
    y='MARGEN_BRUTO_PORCENTAJE',
    text='MARGEN_BRUTO_PORCENTAJE',
    labels={'C_COMPRADOR': 'Comprador', 'MARGEN_BRUTO_PORCENTAJE': 'Margen Bruto Promedio (%)'},
    title='Margen Bruto Promedio por Comprador (Último Mes)'
)

# Formato del texto
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    yaxis_title='Margen Bruto Promedio (%)',
    xaxis_title='Código de Comprador',
    yaxis_tickformat=',.1f'
)

fig.show()

<h3>5. Indicadores de Oportunidades y Riesgos</h3>

In [ ]:
# Promociones activas
# Traer todas las tablas del linked server
query = f"""
SELECT 
    S.C_SUCU_EMPR,                                -- Código de Sucursal
    COUNT(*) AS PROMOCIONES_ACTIVAS,              -- Cantidad de artículos en promoción
    SUM(S.I_PRECIO_VTA) AS VALOR_TOTAL_PROMOCIONES -- Suma de precios de artículos en promoción
FROM 
    [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A ON A.C_ARTICULO = S.C_ARTICULO
WHERE 
    S.M_OFERTA_SUCU = 'S'  -- Solo artículos en promoción
    AND S.M_HABILITADO_SUCU = 'S'  -- Solo artículos habilitados para reposición
    AND A.M_BAJA = 'N'  -- Solo artículos activos
GROUP BY 
    S.C_SUCU_EMPR
ORDER BY 
    PROMOCIONES_ACTIVAS DESC;
"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

df['VALOR_TOTAL_PROMOCIONES'] = df['VALOR_TOTAL_PROMOCIONES'].apply(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
# Crear la tabla en Plotly
fig = go.Figure(data=[go.Table(
    header=dict(
        values=["Sucursal", "Promociones Activas", "Valor Total Promociones"],
        fill_color='lightgrey',
        align='center',
        font=dict(color='black', size=12)
    ),
    cells=dict(
        values=[
            df['C_SUCU_EMPR'],
            df['PROMOCIONES_ACTIVAS'],
            df['VALOR_TOTAL_PROMOCIONES']
        ],
        fill_color='white',
        align='center',
        font=dict(color='black', size=11)
    ))
])

fig.update_layout(
    title="Promociones Activas por Sucursal",
    margin=dict(l=20, r=20, t=40, b=20)
)

fig.show()

In [ ]:
#PRODUCTOS CON BAJA ROTACION
query = f"""
SELECT 
    S.C_ARTICULO,
    A.N_ARTICULO,
    S.C_SUCU_EMPR,
    S.Q_VTA_ULTIMOS_30DIAS,
    S.F_ULTIMA_VTA
FROM 
    [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A 
    ON A.C_ARTICULO = S.C_ARTICULO
WHERE 
    S.Q_VTA_ULTIMOS_30DIAS < 3
    AND S.F_ULTIMA_VTA > '1900-01-01' 
    AND S.M_HABILITADO_SUCU = 'S'
    AND A.M_BAJA = 'N'
ORDER BY 
    S.C_ARTICULO, S.C_SUCU_EMPR;

"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()
#df['N_ARTICULO'] = df['N_ARTICULO'].astype(float).astype(int).astype(str)
# Agrupar: contar en cuántas sucursales está en baja rotación cada artículo
articulo_sucursales = df.groupby(['C_ARTICULO', 'N_ARTICULO']).size().reset_index(name='CANTIDAD_SUCURSALES')

# Ordenar descendente para ver primero los peores
articulo_sucursales = articulo_sucursales.sort_values(by='CANTIDAD_SUCURSALES', ascending=False)
top10articulos = articulo_sucursales.head(15)

# Crear gráfico de barras
fig = px.bar(
    top10articulos,
    x='N_ARTICULO',
    y='CANTIDAD_SUCURSALES',
    labels={
        'N_ARTICULO': 'Artículo',
        'CANTIDAD_SUCURSALES': 'Sucursales en Baja Rotación'
    },
    title='15 articulos con mas sucursales en baja rotacion',
    text='CANTIDAD_SUCURSALES'
)

# Estética
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_title="Artículo",
    yaxis_title="Cantidad de Sucursales",
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    bargap=0.3
)

fig.show()

In [ ]:
#Productos con stock critico
# Traer todas las tablas del linked server
query = f"""
SELECT 
    S.C_ARTICULO,
    S.C_SUCU_EMPR,
    A.N_ARTICULO,
    ST.Q_UNID_ARTICULO AS STOCK_ACTUAL,
    R.Q_DIAS_STOCK
FROM 
    [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A ON A.C_ARTICULO = S.C_ARTICULO
LEFT JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T060_STOCK] ST ON ST.C_ARTICULO = S.C_ARTICULO AND ST.C_SUCU_EMPR = S.C_SUCU_EMPR
LEFT JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T710_ESTADIS_REPOSICION] R ON R.C_ARTICULO = S.C_ARTICULO AND R.C_SUCU_EMPR = S.C_SUCU_EMPR
WHERE 
    ST.Q_UNID_ARTICULO < 5
    AND S.M_HABILITADO_SUCU = 'S'
    AND A.M_BAJA = 'N';
"""
# Leer los datos
df_stock_critico = pd.read_sql_query(query, conn)
conn.close()
#df['N_ARTICULO'] = df['N_ARTICULO'].astype(float).astype(int).astype(str)
# Definir formato condicional: rojo si días de stock < 5, blanco si no
colors = ['red' if dias < 5 else 'white' for dias in df_stock_critico['Q_DIAS_STOCK']]

# Crear la tabla en Plotly
fig = go.Figure(data=[go.Table(
    header=dict(
        values=["Sucursal", "Código de Artículo", "Nombre Artículo", "Stock Actual (Unidades)"],
        fill_color='lightgrey',
        align='center',
        font=dict(color='black', size=12)
    ),
    cells=dict(
        values=[
            df_stock_critico['C_SUCU_EMPR'],
            df_stock_critico['C_ARTICULO'],
            df_stock_critico['N_ARTICULO'],
            df_stock_critico['STOCK_ACTUAL']
        ],
        fill=dict(color=[
            ['white']*len(df_stock_critico),  # Sucursal
            ['white']*len(df_stock_critico),  # Código de Artículo
            ['white']*len(df_stock_critico),  # Nombre de Artículo
            ['white']*len(df_stock_critico),  # Stock Actual
            colors  # Días de Stock (color condicional)
        ]),
        align='center',
        font=dict(color='black', size=11)
    ))
])

fig.update_layout(
    title="Alertas de Stock Crítico por Artículo",
    margin=dict(l=20, r=20, t=40, b=20)
)


In [ ]:
#Ahorros generados
query = f"""
SELECT det.I_PRECIO_COMPRA,
       det.I_PRECIO_LISTA,
	   det.Q_BULTOS_PROV_PED * det.Q_FACTOR_PROV_PED AS QTY_UNID,
       (det.I_PRECIO_LISTA - det.I_PRECIO_COMPRA) * (det.Q_BULTOS_PROV_PED * det.Q_FACTOR_PROV_PED) AS AHORRO_TOTAL,
       cab.U_SUFIJO_OC,
       cab.F_EMISION
FROM
    [DIARCOP001].[DiarcoP].[dbo].[T081_OC_DETA] det
INNER JOIN
    [DIARCOP001].[DiarcoP].[dbo].[T080_OC_CABE] cab
ON
    cab.C_OC = det.C_OC
    AND cab.U_PREFIJO_OC = det.U_PREFIJO_OC
    AND cab.U_SUFIJO_OC = det.U_SUFIJO_OC
WHERE
    cab.F_EMISION >= '2025-01-01'
    AND det.I_PRECIO_LISTA > det.I_PRECIO_COMPRA + 1;
"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()
df['F_EMISION'] = pd.to_datetime(df['F_EMISION'], errors='coerce')
# Crear columna MES para agrupar
df['MES'] = df['F_EMISION'].dt.to_period('M')
df['MES_NOMBRE'] = df['MES'].dt.strftime('%b %Y')  # Ej: "Jan 2025"
df['MES_ORDEN'] = df['MES'].dt.to_timestamp()

# Agrupar por MES y resumir el ahorro
df_mes = df.groupby(['MES_NOMBRE', 'MES_ORDEN'], as_index=False)['AHORRO_TOTAL'].sum()
df_mes['AHORRO_MM'] = (df_mes['AHORRO_TOTAL'] / 1_000_000).round(1)

# Ordenar por fecha para gráfico correcto
df_mes = df_mes.sort_values(by='MES_ORDEN')

# Crear gráfico con barra y línea de tendencia
fig = go.Figure()

# Barras
fig.add_trace(go.Bar(
    x=df_mes['MES_NOMBRE'],
    y=df_mes['AHORRO_MM'],
    name='Ahorro Mensual',
    text=df_mes['AHORRO_MM'].astype(str) + " MM",
    textposition='auto',
    marker_color='royalblue'
))

# Línea de tendencia
fig.add_trace(go.Scatter(
    x=df_mes['MES_NOMBRE'],
    y=df_mes['AHORRO_MM'],
    mode='lines+markers',
    name='Tendencia',
    line=dict(color='darkorange', width=3),
    marker=dict(size=8)
))

# Layout
fig.update_layout(
    title='Ahorro Total por Mes (en Millones)',
    xaxis_title='Mes',
    yaxis_title='Ahorro Total (MM)',
    yaxis_tickformat=',.1f',
    template='plotly_white'
)

fig.show()

<h2> SELECCION DEL PROVEEDOR </h2>

<h3>1. Relacion comercial</h3>

In [ ]:
# Código del proveedor a consultar
codigo_proveedor = 20

conn = pyodbc.connect(conn_str)

# Tu query completa (simulada aquí como un DataFrame)
query = f"""
SELECT [C_PROVEEDOR],
      [N_PROVEEDOR],
      [F_ALTA],
      [F_BAJA],
      [M_ACTIVO],
      [M_BAJA],
      [N_PAIS]
  FROM [DIARCOP001].[DiarcoP].[dbo].[T020_PROVEEDOR]
  WHERE [C_PROVEEDOR] = {codigo_proveedor};
"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

# Función para determinar el estado en base a M_ACTIVO
def determinar_estado(m_activo):
    return 'ESTADO: ACTIVO' if m_activo == 'S' else 'ESTADO: SUSPENDIDO/BLOQUEADO'

# Seleccionamos el proveedor
proveedor = df.iloc[0]
estado = determinar_estado(proveedor['M_ACTIVO'])

# Crear la tarjeta con Plotly
fig = go.Figure(go.Indicator(
    mode = "number",
    title = {"text": f"<b>{proveedor['N_PROVEEDOR']}</b><br>{estado}"},
    domain = {'x': [0, 1], 'y': [0, 1]}
))

fig.update_layout(
    margin=dict(l=20, r=20, t=50, b=20),
    height=200,
    width=1500
)

fig.show()

In [ ]:
# Traer todas las tablas del linked server
query = f"""
SELECT 
    cab.C_PROVEEDOR,
    cab.C_OC,
    cab.U_PREFIJO_OC,
    cab.U_SUFIJO_OC,
    cab.F_EMISION,
    cab.F_ENTREGA,
    cab.C_SITUAC,
    cab.U_DIAS_LIMITE_ENTREGA,
    cab.F_COMP_ING_MERC,
    DATEADD(DAY, cab.U_DIAS_LIMITE_ENTREGA, cab.F_ENTREGA) AS FECHA_ACORDADA,
    det.M_CUMPLIDA_PARCIAL,
    YEAR(cab.F_ENTREGA) AS ANIO_ENTREGA,
    MONTH(cab.F_ENTREGA) AS MES_ENTREGA
FROM 
    [DIARCOP001].[DiarcoP].[dbo].[T080_OC_CABE] cab
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T081_OC_DETA] det
ON 
    cab.C_OC = det.C_OC 
    AND cab.U_PREFIJO_OC = det.U_PREFIJO_OC
    AND cab.U_SUFIJO_OC = det.U_SUFIJO_OC
WHERE 
    YEAR(cab.F_ENTREGA) = 2025
    AND cab.C_PROVEEDOR = {codigo_proveedor}
"""

# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

"""df['C_RUBRO'] = df['C_RUBRO'].astype(float).astype(int).astype(str)
df['C_ARTICULO'] = df['C_ARTICULO'].astype(float).astype(int).astype(str)
df['C_SUCU_EMPR'] = df['C_SUCU_EMPR'].astype(float).astype(int).astype(str)"""

# Convertir fechas
df['F_ENTREGA'] = pd.to_datetime(df['F_ENTREGA'])
df['FECHA_ACORDADA'] = pd.to_datetime(df['FECHA_ACORDADA'])

# --- FILTRAR las OC que aún no llegaron a la fecha pactada ---
mask_pendiente_y_en_tiempo = (df['FECHA_ACORDADA'] >= pd.Timestamp.now())
df = df[~mask_pendiente_y_en_tiempo]

# --- Calcular si estuvo en tiempo y completo ---
df['en_tiempo'] = df['F_ENTREGA'] <= (df['FECHA_ACORDADA'] + pd.Timedelta(days=1))

# AHORA COMPLETO = (entregado completo en detalle) Y (C_SITUAC = 2 en cabecera)
df['completo'] = (df['M_CUMPLIDA_PARCIAL'] == 'N') & (df['C_SITUAC'] == 2)

# --- Agrupar por OC ---
df_oc = df.groupby('U_SUFIJO_OC').agg({
    'en_tiempo': 'all',
    'completo': 'all',
    'ANIO_ENTREGA': 'first',
    'MES_ENTREGA': 'first'
}).reset_index()

# --- Calcular OTIF ---
df_oc['otif'] = df_oc['en_tiempo'] & df_oc['completo']

# --- Agrupar por Mes ---
otif_por_mes = df_oc.groupby(['ANIO_ENTREGA', 'MES_ENTREGA'])['otif'].mean().reset_index()
otif_por_mes['otif'] = otif_por_mes['otif'] * 100  # Convertir a %

# Formato para el gráfico
otif_por_mes['periodo'] = pd.to_datetime(
    otif_por_mes['ANIO_ENTREGA'].astype(str) + '-' + otif_por_mes['MES_ENTREGA'].astype(str).str.zfill(2) + '-01'
)
otif_por_mes['nombre_mes'] = otif_por_mes['MES_ENTREGA'].apply(lambda x: calendar.month_name[x])

#GRAFICO
fig = px.bar(
    otif_por_mes,
    x='nombre_mes',
    y='otif',
    text=otif_por_mes['otif'].apply(lambda x: f"{x:.1f}%"),
    title=f'Cumplimiento OTIF Mensual - Proveedor {codigo_proveedor} <br>',
    labels={'otif': 'Cumplimiento (%)', 'nombre_mes': 'Mes'},
)

fig.update_traces(
    textposition='inside',  # Coloca el texto dentro de la barra
    textfont=dict(size=20, color="white"),  # Aumenta el tamaño del texto y lo hace blanco para mejor visibilidad
    marker_color='rgba(0, 180, 220, 0.7)'  # Mantiene el color de las barras
)

fig.update_layout(
    height=450,
    width=800,
    yaxis_tickformat=".0%",
    xaxis_title='Mes',
    yaxis_title='Cumplimiento (%)',
    title_x=0.5,
    margin=dict(l=20, r=20, t=70, b=50)
)

fig.show()

<h3>2. Promociones e incentivos</h3>

In [ ]:
conn = pyodbc.connect(conn_str)

codigo_proveedor = 62
# Traer todas las tablas del linked server
query = f"""
SELECT 
    o.[C_ANIO],
    o.[C_MES],
    o.[C_SUCU_EMPR],
    o.[C_ARTICULO],
    p.[C_PROVEEDOR],
    a.[N_ARTICULO],  
    o.[M_OFERTA_DIA1],
    o.[M_OFERTA_DIA2],
    o.[M_OFERTA_DIA3],
    o.[M_OFERTA_DIA4],
    o.[M_OFERTA_DIA5],
    o.[M_OFERTA_DIA6],
    o.[M_OFERTA_DIA7],
    o.[M_OFERTA_DIA8],
    o.[M_OFERTA_DIA9],
    o.[M_OFERTA_DIA10],
    o.[M_OFERTA_DIA11],
    o.[M_OFERTA_DIA12],
    o.[M_OFERTA_DIA13],
    o.[M_OFERTA_DIA14],
    o.[M_OFERTA_DIA15],
    o.[M_OFERTA_DIA16],
    o.[M_OFERTA_DIA17],
    o.[M_OFERTA_DIA18],
    o.[M_OFERTA_DIA19],
    o.[M_OFERTA_DIA20],
    o.[M_OFERTA_DIA21],
    o.[M_OFERTA_DIA22],
    o.[M_OFERTA_DIA23],
    o.[M_OFERTA_DIA24],
    o.[M_OFERTA_DIA25],
    o.[M_OFERTA_DIA26],
    o.[M_OFERTA_DIA27],
    o.[M_OFERTA_DIA28],
    o.[M_OFERTA_DIA29],
    o.[M_OFERTA_DIA30],
    o.[M_OFERTA_DIA31]
FROM 
    [DIARCOP001].[DiarcoP].[dbo].[T710_ESTADIS_OFERTA_FOLDER] o
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T052_ARTICULOS_PROVEEDOR] p 
ON 
    o.[C_ARTICULO] = p.[C_ARTICULO]
INNER JOIN 
    [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] a 
ON 
    o.[C_ARTICULO] = a.[C_ARTICULO]
WHERE 
    p.[C_PROVEEDOR] = {codigo_proveedor}
    AND o.[C_ANIO] = 2025;
"""
#WHERE C_PROVEEDOR = {codigo_proveedor};
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

# ------------------------
dias_oferta_cols = [col for col in df.columns if 'M_OFERTA_DIA' in col]
df[dias_oferta_cols] = df[dias_oferta_cols].replace({'S': 1, 'N': 0}).fillna(0)

df_melted = df.melt(
    id_vars=['C_ANIO', 'C_MES', 'C_ARTICULO', 'N_ARTICULO', 'C_SUCU_EMPR', 'C_PROVEEDOR'],
    value_vars=dias_oferta_cols,
    var_name='DIA_MES',
    value_name='EN_OFERTA'
)

df_melted['DIA_MES'] = df_melted['DIA_MES'].str.extract('(\d+)').astype(int)

df_melted = df_melted[df_melted['EN_OFERTA'] == 1]
df_grouped = df_melted.groupby(
    ['C_ANIO', 'C_MES', 'DIA_MES', 'C_ARTICULO', 'N_ARTICULO']
).size().reset_index(name='EN_OFERTA')
dias_en_oferta_por_articulo = df_grouped.groupby(
    ['C_ARTICULO', 'N_ARTICULO']
).size().reset_index(name='dias_en_oferta')

# Cantidad de días transcurridos en 2025
hoy = datetime.now()
inicio_2025 = datetime(2025, 1, 1)
dias_2025 = (hoy - inicio_2025).days
días_pasados_2025 = dias_2025 if dias_2025 > 0 else 0

# Promedio de días en oferta
promedio_dias_oferta = dias_en_oferta_por_articulo['dias_en_oferta'].mean()

fig = px.bar(
    dias_en_oferta_por_articulo.sort_values(by='dias_en_oferta', ascending=False).head(20),
    x='dias_en_oferta',
    y='N_ARTICULO',
    orientation='h',
    title='Cant de Días en Oferta por Artículo (único por día, no por sucursal)',
    labels={'dias_en_oferta': 'Cantidad de días en oferta', 'N_ARTICULO': 'Artículo'}
)

# Agregar anotaciones
fig.add_annotation(
    xref="paper", yref="paper",
    x=0.95, y=1.08,
    text=f"Días transcurridos 2025: {días_pasados_2025}",
    showarrow=False,
    font=dict(size=14, color="blue")
)

fig.add_annotation(
    xref="paper", yref="paper",
    x=0.95, y=1.02,
    text=f"Promedio días en oferta: {promedio_dias_oferta:.1f}",
    showarrow=False,
    font=dict(size=14, color="green")
)

fig.update_layout(
    height=650,
    width=1000,
    yaxis_title='Artículo',
    xaxis_title='Cantidad de días únicos en oferta',
    margin=dict(l=20, r=20, t=80, b=20)
)

fig.show()

In [ ]:
# ------------------------
dias_oferta_cols = [col for col in df.columns if 'M_OFERTA_DIA' in col]
df[dias_oferta_cols] = df[dias_oferta_cols].replace({'S': 1, 'N': 0}).fillna(0)

df_melted = df.melt(
    id_vars=['C_ANIO', 'C_MES', 'C_ARTICULO', 'N_ARTICULO', 'C_SUCU_EMPR', 'C_PROVEEDOR'],
    value_vars=dias_oferta_cols,
    var_name='DIA_MES',
    value_name='EN_OFERTA'
)

df_melted['DIA_MES'] = df_melted['DIA_MES'].str.extract('(\d+)').astype(int)

df_melted = df_melted[df_melted['EN_OFERTA'] == 1]
# 1. Agrupar: artículos únicos por día del mes
articulos_por_dia = df_melted.groupby('DIA_MES')['C_ARTICULO'].nunique().reset_index()
articulos_por_dia.columns = ['dia', 'cantidad_articulos']

# 2. GRAFICO AREA
fig3 = px.area(
    articulos_por_dia.sort_values('dia'),
    x='dia',
    y='cantidad_articulos',
    title='Distribución de Artículos en Oferta por Día del Mes (único por día)',
    labels={'dia': 'Día del Mes', 'cantidad_articulos': 'Cantidad de Artículos en Oferta'}
)

# 3. Opciones de diseño
fig3.update_layout(
    height=400,
    width=800,
    xaxis=dict(tickmode='linear', tick0=1, dtick=1),
    yaxis_title='Cantidad de artículos únicos',
    xaxis_title='Día del mes',
    margin=dict(l=20, r=20, t=40, b=20)
)

fig3.show()

In [ ]:

dias_oferta_cols = [col for col in df.columns if 'M_OFERTA_DIA' in col]
df[dias_oferta_cols] = df[dias_oferta_cols].replace({'S': 1, 'N': 0}).fillna(0)

# --- Crear df_melted ---
df_melted = df.melt(
    id_vars=['C_ANIO', 'C_MES', 'C_ARTICULO', 'N_ARTICULO', 'C_SUCU_EMPR', 'C_PROVEEDOR'],
    value_vars=dias_oferta_cols,
    var_name='DIA_MES',
    value_name='EN_OFERTA'
)

# Extraer número de día del mes
df_melted['DIA_MES'] = df_melted['DIA_MES'].str.extract('(\d+)').astype(int)

# Filtrar solo las filas donde el artículo estuvo en oferta
df_melted = df_melted[df_melted['EN_OFERTA'] == 1]

# --- Agrupar artículos únicos por sucursal ---
articulos_por_sucursal = df_melted.groupby('C_SUCU_EMPR')['C_ARTICULO'].nunique().reset_index()
articulos_por_sucursal.columns = ['sucursal', 'cantidad_articulos']

# Eliminar sucursales sin artículos
articulos_por_sucursal = articulos_por_sucursal[articulos_por_sucursal['cantidad_articulos'] > 0]

# Ordenar de mayor a menor
articulos_por_sucursal = articulos_por_sucursal.sort_values(by='cantidad_articulos', ascending=False)
print(articulos_por_sucursal)
# --- Tomar solo el Top 5 ---
top5_sucursales = articulos_por_sucursal.head(5)
top5_sucursales['sucursal'] = top5_sucursales['sucursal'].astype(str) #para que haga bien la separacion de las columnas
# --- Grafico ---
fig5 = px.bar(
    top5_sucursales,
    x='sucursal',
    y='cantidad_articulos',
    text='cantidad_articulos',
    title='Top 5 Sucursales con más Artículos en Oferta',
    labels={'sucursal': 'Sucursal', 'cantidad_articulos': 'Cantidad de Artículos en Oferta'}
)

fig5.update_traces(textposition='outside')

fig5.update_layout(
    height=400,
    width=700,
    xaxis_title='Sucursal',
    yaxis_title='Cantidad de artículos únicos',
    xaxis_tickangle=0,  # Ya no necesitamos rotar los nombres si son pocos
    xaxis_type='category',  # Forzar eje categórico
    margin=dict(l=20, r=20, t=40, b=40)
)

fig5.show()

<h3>3. Historial de pedidos</h3>

In [ ]:
# Código del proveedor a consultar
codigo_proveedor = 20

conn = pyodbc.connect(conn_str)

# query completa
query = f"""
SELECT [C_OC]
      ,[U_PREFIJO_OC]
      ,[U_SUFIJO_OC]      
      ,[U_DIAS_LIMITE_ENTREGA]
	  , DATEADD(DAY, [U_DIAS_LIMITE_ENTREGA], [F_ENTREGA]) as FECHA_LIMITE
	  , DATEDIFF (DAY, DATEADD(DAY, [U_DIAS_LIMITE_ENTREGA], [F_ENTREGA]), GETDATE()) as DEMORA
      ,[C_PROVEEDOR]
      ,[C_SUCU_COMPRA]
      ,[C_SUCU_DESTINO]
      ,[C_SUCU_DESTINO_ALT]
	  ,[I_TOTAL_OC]
      ,[C_SITUAC]
      ,[F_SITUAC]
      ,[F_ALTA_SIST]
      ,[F_EMISION]
      ,[F_ENTREGA]      
  FROM [DIARCOP001].[DiarcoP].[dbo].[T080_OC_CABE]
  WHERE C_PROVEEDOR = {codigo_proveedor}
"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

# ---------------------
# Última OC Emitida
# ---------------------
ultima_oc = df.sort_values(by='F_EMISION', ascending=False).iloc[0]

fig1 = go.Figure(go.Indicator(
    mode="number",
    value=ultima_oc['I_TOTAL_OC'],
    title={"text": f"<b>Última OC Emitida</b><br>Fecha: {ultima_oc['F_EMISION'].date()}"},
    number={"prefix": "$"}
))

fig1.update_layout(height=250, width=400, margin=dict(t=40, b=0, l=0, r=0))
fig1.show()

hoy = pd.Timestamp.today()
hace_12_meses = hoy - pd.DateOffset(months=12)

df_ultimos_12_meses = df[df['F_EMISION'] >= hace_12_meses]

# ---------------------
# Promedio de OCs por Mes
# ---------------------
df_ultimos_12_meses['Mes'] = df_ultimos_12_meses['F_EMISION'].dt.to_period('M')
ocs_mensuales = df_ultimos_12_meses.groupby('Mes')['I_TOTAL_OC'].sum().reset_index()
ocs_mensuales['Mes'] = ocs_mensuales['Mes'].dt.to_timestamp()

# Promedio de órdenes emitidas por mes
promedio_mensual = ocs_mensuales['I_TOTAL_OC'].mean()

# Gráfico
fig2 = px.bar(ocs_mensuales, x='Mes', y='I_TOTAL_OC', title='Órdenes de Compra por Mes (Últimos 12 meses)')
fig2.update_traces(texttemplate='%{y:.2s}', textposition='outside')
fig2.update_layout(height=400, width=600, yaxis_title='Monto Total OC ($)',
    xaxis=dict(
        tickformat='%b %Y',  # Formato Mes Año
        ),
    yaxis_tickformat='$,.2s',)
fig2.show()

# ---------------------
# Mostrar el Promedio Mensual en una tarjeta
# ---------------------
fig3 = go.Figure(go.Indicator(
    mode="number",
    value=promedio_mensual,
    title={"text": "<b>Promedio Mensual de OCs</b>"},
    number={"prefix": "$"}
))

fig3.update_layout(height=250, width=400, margin=dict(t=40, b=0, l=0, r=0))
fig3.show()

<h3>4. Productos del proveedor</h3>

In [ ]:
# Código del proveedor a consultar
codigo_proveedor = 20

conn = pyodbc.connect(conn_str)
# query completa
query = f"""
SELECT 
    V.F_VENTA,
    V.C_ARTICULO,
    A.N_ARTICULO,
    A.C_PROVEEDOR_PRIMARIO,
    V.I_VENDIDO
FROM 
    [DCO-DBCORE-P02].[DiarcoEst].[dbo].[T702_EST_VTAS_POR_ARTICULO] AS V
INNER JOIN 
    [DCO-DBCORE-P02].[DiarcoEst].[dbo].[T050_ARTICULOS] AS A
ON 
    V.C_ARTICULO = A.C_ARTICULO
WHERE 
    V.F_VENTA >= DATEADD(MONTH, -12, GETDATE())  -- Solo últimos 12 meses
    AND C_PROVEEDOR_PRIMARIO = {codigo_proveedor}
"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()
# ---------------------
# Agrupar y sumar ventas
# ---------------------
df_grouped = df.groupby(['C_ARTICULO', 'N_ARTICULO'])['I_VENDIDO'].sum().reset_index()

# Ordenar por I_VENDIDO descendente
df_top10 = df_grouped.sort_values(by='I_VENDIDO', ascending=False).head(10)

# Redondear a 2 decimales
df_top10['I_VENDIDO'] = df_top10['I_VENDIDO'].round(2)

# ---------------------
# Graficar
fig = px.bar(df_top10, 
             x='I_VENDIDO', 
             y='N_ARTICULO', 
             title='Top 10 Productos más Vendidos (últimos 12 meses)',
             text='I_VENDIDO',
             orientation='h')  

fig.update_layout(
    height=700,  
    width=1000,  
    xaxis_title='Monto Vendido ($)',
    yaxis_title='Producto',
    font=dict(size=14),
    margin=dict(l=200, r=20, t=50, b=50)
)

# Separadores estilo 1.111.111 y dos decimales
fig.update_xaxes(
    tickformat=',.2f'
)
# Mostrar texto dentro de las barras
fig.update_traces(
    texttemplate='%{text:,.2f}',
    textposition='inside'
)
fig.show()

In [ ]:
# Código del proveedor a consultar
codigo_proveedor = 20

conn = pyodbc.connect(conn_str)
# QUERY STOCK DE PRODUCTOS
query = f"""
SELECT A.[C_PROVEEDOR_PRIMARIO]
	,S.[C_ARTICULO]
	,S.[C_SUCU_EMPR]
	,(ST.Q_UNID_ARTICULO + ST.Q_PESO_ARTICULO) AS Q_STOCK_UNIDADES-- Stock Cierre Dia Anterior	
	,A.[N_ARTICULO]
	
        
FROM [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
INNER JOIN [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A
	ON A.[C_ARTICULO] = S.[C_ARTICULO]
LEFT JOIN [DIARCOP001].[DiarcoP].[dbo].[T060_STOCK] ST
	ON ST.C_ARTICULO = S.[C_ARTICULO] 
	AND ST.C_SUCU_EMPR = S.[C_SUCU_EMPR]


WHERE A.[C_PROVEEDOR_PRIMARIO] = {codigo_proveedor} 
        
ORDER BY S.[C_ARTICULO],S.[C_SUCU_EMPR];
"""

# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

df['C_ARTICULO'] = df['C_ARTICULO'].astype(float).astype(int).astype(str)
df['C_SUCU_EMPR'] = df['C_SUCU_EMPR'].astype(float).astype(int).astype(str)

In [ ]:
# ---------------------
# Filtrar productos en nivel crítico
# ---------------------
nivel_critico = df[df['Q_STOCK_UNIDADES'] < 10] #CHEQUEAR CUAL ES NIVEL CRITICO
# Contar cuántos productos en nivel crítico
cantidad_criticos = nivel_critico.shape[0]
# ---------------------
# Crear tarjeta con número de productos críticos
# ---------------------
fig = go.Figure(go.Indicator(
    mode="number",
    value=cantidad_criticos,
    title={"text": "<b>Productos en Nivel Crítico</b>"},
    number={
        "suffix": " articulos",
        "valueformat": ",d"  # <= Mostrar número entero completo
    }
))

fig.update_layout(
    height=250,
    width=1500,
    margin=dict(t=40, b=0, l=0, r=0),
    paper_bgcolor="white"
)

fig.show()

# ---------------------
# Mostrar el listado de productos críticos
# ---------------------
# Seleccionar las columnas necesarias
listado = nivel_critico[['N_ARTICULO', 'Q_STOCK_UNIDADES', 'C_SUCU_EMPR']]
# Mostrar tabla
fig2 = go.Figure(data=[go.Table(
    header=dict(
        values=["Artículo", "Cantidad", "Sucursal"],
        fill_color='lightblue',
        align='left',
        font=dict(size=16)  # Más grande el texto del encabezado
    ),
    cells=dict(
        values=[
            listado['N_ARTICULO'],
            listado['Q_STOCK_UNIDADES'],
            listado['C_SUCU_EMPR']
        ],
        align='left',
        font=dict(size=14)  # Más grande el texto de las filas
    )
)])

fig2.update_layout(
    height=800,   # MÁS ALTO
    width=1500,   # MÁS ANCHO
    title="Listado de Productos en Nivel Crítico",
    title_font_size=22,
    margin=dict(l=20, r=20, t=50, b=20)
)

fig2.show()

In [ ]:
df.info()

<h3>5. Rendimiento del proveedor</h3>

In [ ]:
# Código del proveedor a consultar
codigo_proveedor = 20
conn = pyodbc.connect(conn_str)

# query completa
query = f"""
SELECT [C_OC]
      ,[U_PREFIJO_OC]
      ,[U_SUFIJO_OC]      
      ,[U_DIAS_LIMITE_ENTREGA]
	  , DATEADD(DAY, [U_DIAS_LIMITE_ENTREGA], [F_ENTREGA]) as FECHA_LIMITE
	  , DATEDIFF (DAY, DATEADD(DAY, [U_DIAS_LIMITE_ENTREGA], [F_ENTREGA]), GETDATE()) as DEMORA
      ,[C_PROVEEDOR]
      ,[C_SUCU_COMPRA]
      ,[C_SUCU_DESTINO]
      ,[C_SUCU_DESTINO_ALT]
	  ,[I_TOTAL_OC]
      ,[C_SITUAC]
      ,[F_SITUAC]
      ,[F_ALTA_SIST]
      ,[F_EMISION]
      ,[F_ENTREGA]    
       ,[C_USUARIO_OPERADOR]    
      
  FROM [DIARCOP001].[DiarcoP].[dbo].[T080_OC_CABE]  
  WHERE C_PROVEEDOR = {codigo_proveedor};
"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()
# ---------------------
# 1. Calcular Tiempo Promedio de Entrega
# ---------------------

df['F_EMISION'] = pd.to_datetime(df['F_EMISION'], errors='coerce')
df['F_ENTREGA'] = pd.to_datetime(df['F_ENTREGA'], errors='coerce')
# ---------------------
# FILTRAR: Excluir C_SITUAC = 3
# ---------------------
df_filtrado = df[df['C_SITUAC'] != 3 & 1].copy()

df_filtrado['dias_entrega'] = (df_filtrado['F_ENTREGA'] - df_filtrado['F_EMISION']).dt.days

# Tiempo promedio
promedio_entrega = df_filtrado['dias_entrega'].mean()

# Mostrar tiempo promedio
fig = go.Figure(go.Indicator(
    mode="number",
    value=promedio_entrega,
    title={"text": "<b>Tiempo Promedio de Entrega (días)</b>"},
    number={"valueformat": ".2f"}  # 2 decimales
))

fig.update_layout(
    height=250,
    width=400,
    margin=dict(t=40, b=0, l=0, r=0),
    paper_bgcolor="white"
)

fig.show()

# ---------------------
# 2. Pie Chart de Día de la Semana
# ---------------------
# Sacamos día de la semana
df_filtrado['dia_semana'] = df_filtrado['F_ENTREGA'].dt.day_name()

# Agrupamos
dias_semana = df_filtrado['dia_semana'].value_counts().reset_index()
dias_semana.columns = ['Día', 'Cantidad']

# Pie chart
fig2 = px.pie(dias_semana, values='Cantidad', names='Día', title='Entregas por Día de la Semana')
fig2.update_traces(textposition='inside', textinfo='percent+label')

fig2.update_layout(
    height=500,
    width=500
)

fig2.show()

# ---------------------
# Contar pendientes y anulados
# ---------------------
"""cantidad_pendientes = (df['C_SITUAC'] == 1).count()
cantidad_anulados = (df['C_SITUAC'] == 3).count()"""

cantidad_pendientes = (df['C_SITUAC'] == 1).sum()
cantidad_anulados = (df['C_SITUAC'] == 3).sum()
# ---------------------
# Tarjeta de Pendientes
# ---------------------
fig_pendientes = go.Figure(go.Indicator(
    mode="number",
    value=cantidad_pendientes,
    title={"text": "<b>Órdenes Pendientes</b>"},
    number={"valueformat": ",d"}  # Mostrar número completo
))

fig_pendientes.update_layout(
    height=250,
    width=400,
    margin=dict(t=40, b=0, l=0, r=0),
    paper_bgcolor="white"
)

fig_pendientes.show()

# ---------------------
# Tarjeta de Anulados
# ---------------------
fig_anulados = go.Figure(go.Indicator(
    mode="number",
    value=cantidad_anulados,
    title={"text": "<b>Órdenes Anuladas</b>"},
    number={"valueformat": ",d"}
))

fig_anulados.update_layout(
    height=250,
    width=400,
    margin=dict(t=40, b=0, l=0, r=0),
    paper_bgcolor="white"
)

fig_anulados.show()

<h3>6. Resumen financiero</h3>

In [ ]:
conn = pyodbc.connect(conn_str)
codigo_proveedor = 189
# Traer todas las tablas del linked server
query = f"""
SELECT I_SALDO
FROM [DIARCOP001].[DiarcoP].[dbo].[T999_SALDOS_PROVEEDOR]
WHERE C_PROVEEDOR = {codigo_proveedor};
"""
# Leer los datos
df = pd.read_sql_query(query, conn)
conn.close()

# ----------------------
saldo = df['I_SALDO'].iloc[0]
# Tarjeta: Deuda Total
fig_deuda = go.Figure(go.Indicator(
    mode="number",
    value=saldo,
    title={"text": "<b>Deuda Actual Total</b>"},
    number={"valueformat": ",.2f", "prefix": "$"}
))
fig_deuda.update_layout(height=250, width=400, paper_bgcolor="white")
fig_deuda.show()